In [25]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
import nltk
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import gensim.downloader as api
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
#fork

## Project 1 - NLP and Text Classification

For this project you will need to classify some angry comments into their respective category of angry. The process that you'll need to follow is (roughly):
<ol>
<li> Use NLP techniques to process the training data. 
<li> Train model(s) to predict which class(es) each comment is in.
    <ul>
    <li> A comment can belong to any number of classes, including none. 
    </ul>
<li> Generate predictions for each of the comments in the test data. 
<li> Write your test data predicitions to a CSV file, which will be scored. 
</ol>

You can use any models and NLP libraries you'd like. Think aobut the problem, look back to see if there's anything that might help, give it a try, and see if that helps. We've regularly said we have a "toolkit" of things that we can use, we generally don't know which ones we'll need, but here you have a pretty simple goal - if it makes it more accurate, it helps. There's not one specific solution here, there are lots of things that you could do. 

## Training Data

Use the training data to train your prediction model(s). Each of the classification output columns (toxic to the end) is a human label for the comment_text, assessing if it falls into that category of "rude". A comment may fall into any number of categories, or none at all. Membership in one output category is <b>independent</b> of membership in any of the other classes (think about this when you plan on how to make these predictions - it may also make it easier to split work amongst a team...). 

In [3]:
#MULTI LABEL

In [4]:
train_df = pd.read_csv("train.csv.zip")
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


#### Function Creation and Setup

In [5]:

# nltk.download('all')
# for package in ['stopwords','punkt','wordnet']:
#     nltk.download(package) 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 

In [6]:
# class lemmaTokenizer(object):
#     def __init__(self, stop_words):
#         self.stop_words = stop_words
#         self.lemmatizer = WordNetLemmatizer()
#     def __call__(self, doc):
#         tokens = word_tokenize(doc)
#         filtered_tok = []
#         for tok in tokens:
#             if tok not in stop_words:
#                 tok = re.sub('\W+','', tok) #Punctuation strip
#                 tmp = self.lemmatizer.lemmatize(tok)
#                 if len(tmp) >= 2:
#                     filtered_tok.append(tmp)
#         return filtered_tok

In [7]:
#tok = lemmaTokenizer(stop_words)
#train_df["clean_text"] = train_df["comment_text"].apply(lambda x: tok(x))

In [8]:
toxic = train_df[["id","comment_text","toxic"]]

In [9]:
sToxic = train_df[["id","comment_text","severe_toxic"]]

In [10]:
obscene = train_df[["id","comment_text","obscene"]]

In [11]:
threat = train_df[["id","comment_text","threat"]]

In [12]:
idHate = train_df[["id","comment_text","identity_hate"]]

In [13]:
insult = train_df[["id","comment_text","insult"]]

#### Model Creation

In [14]:
#modelTwitter =  api.load("glove-twitter-25")

In [15]:
# w2vTwitter = dict(zip(modelTwitter.index_to_key, modelTwitter.vectors))

# #for converting sentence to vectors/numbers from word vectors result by Word2Vec
# class MeanEmbeddingVectorizer(object):
#     def __init__(self, word2vec):
#         self.word2vec = word2vec
#         # if a text is empty we should return a vector of zeros
#         # with the same dimensionality as all the other vectors
#         self.dim = len(next(iter(word2vec.values())))

#     def fit(self, X, y):
#         return self

#     def transform(self, X):
#         return np.array([
#             np.mean([self.word2vec[w] for w in words if w in self.word2vec]
#                     or [np.zeros(self.dim)], axis=0)
#             for words in X
#         ])

In [18]:
#twitterEmbedded = MeanEmbeddingVectorizer(w2vTwitter)
twitterEmbedded = TfidfVectorizer(max_features=5000,stop_words='english')
ctext = twitterEmbedded.fit_transform(train_df['comment_text'])

# Problems
- Clean the text of punctuation

# Toxic

In [29]:
# Split data - using the new dataframe parts that we cleaned up. 
toxic = train_df[["id","comment_text","toxic"]]
X_train, X_test, y_train, y_test = train_test_split(ctext,toxic["toxic"])

#Word2vec
#X_train_vectors_w2v = twitterEmbedded.transform(X_train)
#X_test_vectors_w2v = twitterEmbedded.transform(X_test)

#oversampling
#sampler = SMOTETomek(n_jobs=-1)
#X_trainSamp, y_trainSamp = sampler.fit_resample(X_train,y_train)

#model fitting
toxicModel = LogisticRegression(n_jobs=-1)
toxicModel.fit(X_train, y_train)

#Predict y value for test dataset
y_predict = toxicModel.predict(X_test)
y_prob = toxicModel.predict_proba(X_test)[:,1]
 
#calculate scores
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)  

print(classification_report(y_test, y_predict))
sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

KeyboardInterrupt: 

# Severely Toxic

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
 
# Split data - using the new dataframe parts that we cleaned up. 
# X_train, X_test, y_train, y_test = train_test_split(sToxic["clean_text"],sToxic["severe_toxic"])

# #Word2vec
# # Fit and transform
# severeMeanEmbeddingVectorizer = MeanEmbeddingVectorizer(w2vToxic)

# X_train_vectors_w2v = severeMeanEmbeddingVectorizer.transform(X_train)
# X_test_vectors_w2v = severeMeanEmbeddingVectorizer.transform(X_test)

# severeModel = RandomForestClassifier()
# severeModel.fit(X_train_vectors_w2v, y_train)  #model

# #Predict y value for test dataset
# y_predict = severeModel.predict(X_test_vectors_w2v)
# y_prob = severeModel.predict_proba(X_test_vectors_w2v)[:,1]
 
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(y_test, y_predict))
# sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

# Obscene

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
 
# Split data - using the new dataframe parts that we cleaned up. 
# X_train, X_test, y_train, y_test = train_test_split(obscene["clean_text"],obscene["obscene"])

# #Word2vec
# # Fit and transform
# obsceneMeanEmbeddingVectorizer = MeanEmbeddingVectorizer(w2vToxic)

# X_train_vectors_w2v = obsceneMeanEmbeddingVectorizer.transform(X_train)
# X_test_vectors_w2v = obsceneMeanEmbeddingVectorizer.transform(X_test)

# obsceneModel = RandomForestClassifier()
# obsceneModel.fit(X_train_vectors_w2v, y_train)  #model

# #Predict y value for test dataset
# y_predict = obsceneModel.predict(X_test_vectors_w2v)
# y_prob = obsceneModel.predict_proba(X_test_vectors_w2v)[:,1]
 
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(y_test, y_predict))
# sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

# Insult

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
 
# Split data - using the new dataframe parts that we cleaned up. 
# X_train, X_test, y_train, y_test = train_test_split(insult["clean_text"],insult["insult"])

# #Word2vec
# # Fit and transform
# insultMeanEmbeddingVectorizer = MeanEmbeddingVectorizer(w2vToxic)

# X_train_vectors_w2v = insultMeanEmbeddingVectorizer.transform(X_train)
# X_test_vectors_w2v = insultMeanEmbeddingVectorizer.transform(X_test)

# insultModel = RandomForestClassifier()
# insultModel.fit(X_train_vectors_w2v, y_train)  #model

# #Predict y value for test dataset
# y_predict = insultModel.predict(X_test_vectors_w2v)
# y_prob = insultModel.predict_proba(X_test_vectors_w2v)[:,1]
 
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(y_test, y_predict))
# sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

# Identity Hate

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
 
# Split data - using the new dataframe parts that we cleaned up. 
# X_train, X_test, y_train, y_test = train_test_split(idHate["clean_text"],idHate["identity_hate"])

# #Word2vec
# # Fit and transform
# idHateMeanEmbeddingVectorizer = MeanEmbeddingVectorizer(w2vToxic)

# X_train_vectors_w2v = idHateMeanEmbeddingVectorizer.transform(X_train)
# X_test_vectors_w2v = idHateMeanEmbeddingVectorizer.transform(X_test)

# idHatePredictionModel = RandomForestClassifier()
# idHatePredictionModel.fit(X_train_vectors_w2v, y_train)  #model

# #Predict y value for test dataset
# y_predict = idHatePredictionModel.predict(X_test_vectors_w2v)
# y_prob = idHatePredictionModel.predict_proba(X_test_vectors_w2v)[:,1]
 
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(y_test, y_predict))
# sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

# Threat

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
 
# Split data - using the new dataframe parts that we cleaned up. 
# X_train, X_test, y_train, y_test = train_test_split(threat["clean_text"],threat["threat"])

# #Word2vec
# # Fit and transform
# threatVectorizeModel = MeanEmbeddingVectorizer(w2vToxic)

# X_train_vectors_w2v = threatVectorizeModel.transform(X_train)
# X_test_vectors_w2v = threatVectorizeModel.transform(X_test)

# threatPredictionModel = RandomForestClassifier()
# threatPredictionModel.fit(X_train_vectors_w2v, y_train)  #model

# #Predict y value for test dataset
# y_predict = threatPredictionModel.predict(X_test_vectors_w2v)
# y_prob = threatPredictionModel.predict_proba(X_test_vectors_w2v)[:,1]
 
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(y_test, y_predict))
# sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

## Test Data

In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

In [ ]:
test_df["clean_text"] = test_df["comment_text"].apply(lambda x: tok(x))

In [ ]:
#Toxic final prediction
# test_vectors_toxic = toxicMeanEmbedVectorizer.transform(test_df["clean_text"])
# yToxic = toxicModel.predict(test_vectors_toxic)

# #Severely toxic final prediction
# test_vectors_severe = severeMeanEmbeddingVectorizer.transform(test_df["clean_text"])
# ySevere = severeModel.predict(test_vectors_toxic)

# #Obscene final prediction
# test_vectors_obscene = obsceneMeanEmbeddingVectorizer.transform(test_df["clean_text"])
# yObscene = obsceneModel.predict(test_vectors_toxic)

# #Insult final prediction
# test_vectors_insult = insultMeanEmbeddingVectorizer.transform(test_df["clean_text"])
# yInsult = insultModel.predict(test_vectors_toxic)

# #Identity Hate final prediction
# test_vectors_idhate = idHateMeanEmbeddingVectorizer.transform(test_df["clean_text"])
# yIDHate = idHatePredictionModel.predict(test_vectors_toxic)

# #Threat final prediction
# test_vectors_threat = threatVectorizeModel.transform(test_df["clean_text"])
# yThreat = threatPredictionModel.predict(test_vectors_toxic)

In [ ]:
#now kith
# columns = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# thisIsGoingOut = pd.DataFrame(list(zip(test_df["id"], yToxic, ySevere,yObscene, yThreat, yInsult, yIDHate)),
#                               columns=columns)
# thisIsGoingOut.to_csv('out.csv', index=False)  

## Output Details, Submission Info, and Example Submission

For this project, please output your predictions in a CSV file. The structure of the CSV file should match the structure of the example below. 

The output should contain one row for each row of test data, complete with the columns for ID and each classification.

Into Moodle please submit:
<ul>
<li> Your notebook file(s). I'm not going to run them, just look. 
<li> Your sample submission CSV. This will be evaluated for accuracy against the real labels; only a subset of the predictions will be scored. 
</ul>

It is REALLY, REALLY, REALLY important the the structure of your output matches the specifications. The accuracies will be calculated by a script, and it is expecting a specific format. 

### Sample Evaluator

The file prediction_evaluator.ipynb contains an example scoring function, scoreChecker. This function takes a sumbission and an answer key, loops through, and evaluates the accuracy. You can use this to verify the format of your submission. I'm going to use the same function to evaluate the accuracy of your submission, against the answer key (unless I made some mistake in this counting function).

In [ ]:
#Construct dummy data for a sample output. 
#You won't do this part first, you have real data - I'm faking it. 
#Your data should have the same structure, so the CSV output is the same
# dummy_ids = ["dfasdf234", "asdfgw43r52", "asdgtawe4", "wqtr215432"]
# dummy_toxic = [0,0,0,0]
# dummy_severe = [0,0,0,0]
# dummy_obscene = [0,1,1,0]
# dummy_threat = [0,1,0,1]
# dummy_insult = [0,0,1,0]
# dummy_ident = [0,1,1,0]
# columns = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# sample_out = pd.DataFrame( list(zip(dummy_ids, dummy_toxic, dummy_severe, dummy_obscene, dummy_threat, dummy_insult, dummy_ident)),
#                     columns=columns)
# sample_out.head()

In [ ]:
#Write DF to CSV. Please keep the "out.csv" filename. Moodle will auto-preface it with an identifier when I download it. 
#This command should work with your dataframe of predictions. 
# sample_out.to_csv('out.csv', index=False)  

## Grading

The grading for this is split between accuracy and well written code:
<ul>
<li> 75% - Accuracy. The most accurate will get 100% on this, the others will be scaled down from there. 
<li> 25% - Code quality. Can the code be followed and made sense of - i.e. comments, sections, titles. 
</ul>